In [4]:
import pandas as pd
import numpy as np
from scipy import spatial
import geopy.distance
import requests
import json
import geopy.distance
import timeit
from datetime import datetime
from PvGis import PvGis
import numpy as np
import pandas as pd
import json
import matplotlib
import rasterio
from rasterio.plot import show
import rioxarray as rxr
from osgeo import gdal
import csv
import math
import sys
import geopandas as gpd
import networkx as nx
import descartes
from shapely.geometry import Point, Polygon
import plotly.graph_objects as go
import plotly.express as px

sys.path.append("C:/Users/Jason/Documents/Papers/mapping paper/model/shapefile_to_network/main/convertor")
sys.path.append("C:/Users/Jason/Documents/Papers/mapping paper/model/shapefile_to_network/main/shortest_paths")

from shapefile_to_network.main.convertor.GraphSimplify import GraphSimplify
from shapefile_to_network.main.convertor.GraphConvertor import GraphConvertor
from shapefile_to_network.main.shortest_paths.ShortestPath import ShortestPath
from shapefile_to_network.main.metrics.Centrality import Centrality

from shapely import speedups
speedups.disable()

In [2]:
df = pd.read_csv('Data/renewables.csv', index_col=0)
df

,Longitude,Latitude,Solar Energy Potential,Wind Power Density,Port Code,Port Long.,Port Lat.,Gen-Port Direct Dist.,Gen-Port Driving Dist.
0,-159.0,56.0,1063.974986,472.33026,4944,-152.415736,57.783555,447.504518,491.8272
1,-159.0,59.0,1079.305006,369.35553,4944,-152.415736,57.783555,408.020243,NaN
2,-135.0,57.0,917.975038,958.34094,3146,-130.000127,55.902870,331.533877,1852.1911
3,-128.0,55.0,999.004996,1433.11040,3679,-128.671822,53.977895,121.818806,207.8273
4,-126.0,55.0,1163.984960,77.40454,3679,-128.671822,53.977895,207.160715,422.9948
...,...,...,...,...,...,...,...,...,...
5965,174.0,-36.0,1325.315015,343.84467,2198,174.539025,-35.865232,50.888321,92.2879
5966,175.0,-39.0,1227.860030,146.18901,2488,174.652134,-38.174366,96.532935,199.0102
5967,175.0,-38.0,1220.925010,331.73700,2488,174.652134,-38.174366,36.137437,54.9704
5968,175.0,-37.0,1302.685002,310.05020,6825,174.767247,-36.840815,27.243295,34.9165


In [17]:
def create_network():

    # Convert the created line shapefile into network using GraphConvertor.py

    # Create GraphConvertor object by passing the path of input shapefile and the output directory
    input_file = 'Data/shipping/shipping_routes/shipping_routes.shp'
    output_dir = 'Data/shipping/nodes'

    graph_convertor_obj = GraphConvertor(input_file, output_dir)

    # Call graph_convertor function to convert the input shapefile into road network and save the newly created shapefile
    # into specifed output_dir along with list of nodes and edges in .csv files
    network = graph_convertor_obj.graph_convertor()

    return network

def shipping_distance(g, shortest_path_obj, alpha, graph_buffer, point_buffer, break_point, start_tuple, end_tuple):

    # Find number of shortest paths from origin to destination in new simplified network

    shortest_paths, buffered_graph = shortest_path_obj.find_shortest_paths(start_tuple, end_tuple)
    shortest_dis = min(shortest_paths.keys())

    return shortest_dis

def create_path(df, end_tuple):
    g = create_network()

    alpha = 0.1
    graph_buffer = 300
    point_buffer = 1
    break_point = 1  # Upper limit to save computation time

    # Create ShortestPath object by passing all required parameters
    shortest_path_obj = ShortestPath(g, alpha, graph_buffer, point_buffer, break_point)

    for i in range(len(df)):
        df['Shipping Distance'][i] = shipping_distance(g, shortest_path_obj, alpha, graph_buffer, point_buffer, break_point,
                                                       (df['Port Long.'][i], df['Port Lat.'][i]), end_tuple)

    return df


In [5]:
#Create port to port distance matrix
df_driving_index = df[['Port Code']].copy()
df_driving_index.to_csv('Data/Driving_index.csv')

In [37]:
#Check for port code and coordinates for table
df_test = df[df['Port Lat.'] < 42]
df_test = df_test[df_test['Port Lat.'] > 38]
df_test = df_test[df_test['Port Long.'] < -72]
df_test = df_test[df_test['Port Long.'] > -76]
df_test

,Longitude,Latitude,Solar Energy Potential,Wind Power Density,Port Code,Port Long.,Port Lat.,Gen-Port Direct Dist.,Gen-Port Driving Dist.
870,-76.0,40.0,1430.800028,257.44534,3493,-75.375989,39.822125,56.89360370277016 km,75.2555
871,-76.0,41.0,1356.705003,543.63580,3493,-75.375989,39.822125,141.1105426672925 km,181.2025
893,-74.0,42.0,1412.549958,116.65230,4201,-73.963834,41.927140,8.630255826882161 km,11.9309


In [38]:
end_tuple = (41.927140, -73.963834) #lat, long

df_port_index['4201'] = np.zeros(len(df_port_index))

g = create_network()

alpha = 0.1
graph_buffer = 300
point_buffer = 1
break_point = 1  # Upper limit to save computation time

# Create ShortestPath object by passing all required parameters
shortest_path_obj = ShortestPath(g, alpha, graph_buffer, point_buffer, break_point)


#start timer
start = timeit.default_timer()

for i in range(len(df)):
    df_port_index['4201'][i] = shipping_distance(g, shortest_path_obj, alpha, graph_buffer, point_buffer, break_point,
                                                       (df['Port Long.'][i], df['Port Lat.'][i]), end_tuple)

    
#stop timer
stop = timeit.default_timer()

print('Time: ', stop - start)

C:/Users/Jason Collis/Documents/Papers/mapping paper/model/shapefile_to_network/main/shortest_paths\BufferedGraph.py:32: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  gdf.crs = {'init': 'epsg:4326'}
c:\users\jason collis\appdata\local\programs\python\python39\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:/Users/Jason Collis/Documents/Papers/mapping paper/model/shapefile_to_network/main/shortest_paths\BufferedGraph.py:15: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  gdf.crs = to_crs
<ipython-input-38-83d823a3e

Time:  3890.1649234000015


In [39]:
df_port_index.to_csv('Data/port_index.csv')
df_port_index

,Port Code,2832,2625,2100,4201
0,4944,15696.189539,13574.376207,9028.136360,16757.084547
1,4944,15696.189539,13574.376207,9028.136360,16757.084547
2,3146,15696.189539,13574.376207,9028.136360,16757.084547
3,3679,14981.975145,13435.420770,9275.676537,16042.870153
4,3679,14981.975145,13435.420770,9275.676537,16042.870153
...,...,...,...,...,...
5965,2198,5732.531650,7891.307073,15679.191090,5600.660317
5966,2488,5732.531650,7891.307073,15679.191090,5600.660317
5967,2488,5732.531650,7891.307073,15679.191090,5600.660317
5968,6825,5732.531650,7891.307073,15679.191090,5600.660317


In [ ]:
54